<a href="https://colab.research.google.com/github/Anjasfedo/eceg-lsb-lzw-huffman/blob/main/HUFFMAN/huffman_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import heapq
from collections import Counter


class HuffmanNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq


class HuffmanEncoding:
    @staticmethod
    def build_frequency_table(text):
        """Build a frequency table for the given text."""
        return Counter(text)

    @staticmethod
    def build_huffman_tree(freq_table):
        """Build the Huffman Tree based on the frequency table."""
        heap = [HuffmanNode(char, freq) for char, freq in freq_table.items()]
        heapq.heapify(heap)

        while len(heap) > 1:
            left = heapq.heappop(heap)
            right = heapq.heappop(heap)
            merged = HuffmanNode(None, left.freq + right.freq)
            merged.left = left
            merged.right = right
            heapq.heappush(heap, merged)

        return heap[0] if heap else None

    @staticmethod
    def generate_huffman_codes(node, prefix='', codebook=None):
        """Recursively generate Huffman codes for each character."""
        if codebook is None:
            codebook = {}
        if node is None:
            return codebook

        if node.char is not None:
            codebook[node.char] = prefix
        else:
            HuffmanEncoding.generate_huffman_codes(
                node.left, prefix + '0', codebook)
            HuffmanEncoding.generate_huffman_codes(
                node.right, prefix + '1', codebook)

        return codebook

    @staticmethod
    def encode(text, codebook):
        """Encode the input text using the Huffman codebook and return as characters."""
        # Convert text to bitstring
        bitstring = ''.join(codebook[char] for char in text)

        # Add padding to make bitstring a multiple of 8
        padding = (8 - len(bitstring) % 8) % 8
        bitstring += '0' * padding

        # Convert bitstring to characters
        char_encoded = ''.join(
            chr(int(bitstring[i:i+8], 2)) for i in range(0, len(bitstring), 8))

        return char_encoded, padding

    @staticmethod
    def decode(encoded_text, huffman_tree, padding):
        """Decode the encoded characters back to the original text."""
        # Convert characters back to bitstring
        bitstring = ''.join(f'{ord(char):08b}' for char in encoded_text)

        # Remove padding
        if padding > 0:
            bitstring = bitstring[:-padding]

        decoded_text = []
        node = huffman_tree

        for bit in bitstring:
            node = node.left if bit == '0' else node.right
            if node.char is not None:
                decoded_text.append(node.char)
                node = huffman_tree

        return ''.join(decoded_text)

    @staticmethod
    def build_huffman(text):
        """Build the Huffman tree, generate codes, and encode the text."""
        freq_table = HuffmanEncoding.build_frequency_table(text)
        huffman_tree = HuffmanEncoding.build_huffman_tree(freq_table)
        codebook = HuffmanEncoding.generate_huffman_codes(huffman_tree)
        encoded_text, padding = HuffmanEncoding.encode(text, codebook)
        return encoded_text, codebook, huffman_tree, padding

    @staticmethod
    def get_compression_ratio(input_string, encoded_text):
        """Calculate the compression ratio."""
        original_bits = len(input_string) * 8
        compressed_bits = len(encoded_text) * 8
        return ((original_bits - compressed_bits) / original_bits) * 100 if original_bits else 0


In [3]:
import unittest


class TestHuffmanEncoding(unittest.TestCase):
    def test_build_frequency_table(self):
        """Test the building of frequency table from text."""
        text = "hello"
        freq_table = HuffmanEncoding.build_frequency_table(text)
        expected_freq = {'h': 1, 'e': 1, 'l': 2, 'o': 1}

        self.assertEqual(freq_table, expected_freq, f"Expected frequency table: {expected_freq}, but got: {freq_table}")

    def test_build_huffman_tree(self):
        """Test the construction of the Huffman Tree."""
        text = "hello"
        freq_table = HuffmanEncoding.build_frequency_table(text)
        huffman_tree = HuffmanEncoding.build_huffman_tree(freq_table)

        self.assertIsNotNone(huffman_tree, "Huffman tree root should not be None.")
        self.assertIsNone(huffman_tree.char, "Root node should be an internal node, not a leaf.")

    def test_generate_huffman_codes(self):
        """Test the generation of Huffman codes."""
        text = "hello"
        freq_table = HuffmanEncoding.build_frequency_table(text)
        huffman_tree = HuffmanEncoding.build_huffman_tree(freq_table)
        codebook = HuffmanEncoding.generate_huffman_codes(huffman_tree)

        # Ensure that all characters have a code
        self.assertGreater(len(codebook), 0, "Codebook should not be empty.")
        for char in text:
            self.assertIn(char, codebook, f"Character {char} missing from codebook.")

    def test_encode(self):
        """Test the encoding of the text."""
        text = "hello"
        freq_table = HuffmanEncoding.build_frequency_table(text)
        huffman_tree = HuffmanEncoding.build_huffman_tree(freq_table)
        codebook = HuffmanEncoding.generate_huffman_codes(huffman_tree)

        encoded_text, padding = HuffmanEncoding.encode(text, codebook)

        self.assertGreater(len(encoded_text), 0, "Encoded text should not be empty.")
        self.assertTrue(0 <= padding < 8, f"Padding should be between 0 and 7, but got {padding}.")

    def test_decode(self):
        """Test the decoding of the encoded text."""
        text = "hello"
        freq_table = HuffmanEncoding.build_frequency_table(text)
        huffman_tree = HuffmanEncoding.build_huffman_tree(freq_table)
        codebook = HuffmanEncoding.generate_huffman_codes(huffman_tree)

        encoded_text, padding = HuffmanEncoding.encode(text, codebook)
        decoded_text = HuffmanEncoding.decode(encoded_text, huffman_tree, padding)

        self.assertEqual(decoded_text, text, f"Decoded text: {decoded_text} does not match original text: {text}.")

    def test_build_huffman(self):
        """Test the full Huffman encoding pipeline."""
        text = "hello"
        encoded_text, codebook, huffman_tree, padding = HuffmanEncoding.build_huffman(text)

        self.assertGreater(len(encoded_text), 0, "Encoded text should not be empty.")
        self.assertIsNotNone(huffman_tree, "Huffman tree should not be None.")
        self.assertTrue(0 <= padding < 8, f"Padding should be between 0 and 7, but got {padding}.")
        self.assertGreater(len(codebook), 0, "Codebook should not be empty.")

    def test_get_compression_ratio(self):
        """Test the compression ratio calculation."""
        text = "hello"
        encoded_text, codebook, huffman_tree, padding = HuffmanEncoding.build_huffman(text)

        compression_ratio = HuffmanEncoding.get_compression_ratio(text, encoded_text)
        self.assertGreater(compression_ratio, 0, f"Expected positive compression ratio, but got {compression_ratio}.")

    def test_decoding_with_bundle(self):
        """Test decoding using bundled Huffman tree and padding."""
        text = "hello"
        encoded_text, codebook, huffman_tree, padding = HuffmanEncoding.build_huffman(text)

        # Simulate bundling tree and padding
        bundle = {"huffman_tree": huffman_tree, "padding": padding}

        # Decode using bundle
        decoded_text = HuffmanEncoding.decode(encoded_text, bundle["huffman_tree"], bundle["padding"])
        self.assertEqual(decoded_text, text, f"Decoded text: {decoded_text} does not match original text: {text}.")


if __name__ == "__main__":
    unittest.main(argv=[""], verbosity=2, exit=False)


test_build_frequency_table (__main__.TestHuffmanEncoding)
Test the building of frequency table from text. ... ok
test_build_huffman (__main__.TestHuffmanEncoding)
Test the full Huffman encoding pipeline. ... ok
test_build_huffman_tree (__main__.TestHuffmanEncoding)
Test the construction of the Huffman Tree. ... ok
test_decode (__main__.TestHuffmanEncoding)
Test the decoding of the encoded text. ... ok
test_decoding_with_bundle (__main__.TestHuffmanEncoding)
Test decoding using bundled Huffman tree and padding. ... ok
test_encode (__main__.TestHuffmanEncoding)
Test the encoding of the text. ... ok
test_generate_huffman_codes (__main__.TestHuffmanEncoding)
Test the generation of Huffman codes. ... ok
test_get_compression_ratio (__main__.TestHuffmanEncoding)
Test the compression ratio calculation. ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.042s

OK
